In [4]:
# test data frame
# from dataframe_excel_charting import DataFrameExcelCharting
import pandas as pd
import numpy as np
# import xlsxwriter
# import copy
import itertools

In [5]:
import pandas as pd
df = pd.DataFrame(columns=['key','pct','brand', 'settlement_id', 'POP'])
df.loc[0] = ['ALL|2G',1,'Movistar','PER_5_216:1',960.03030000001]
df.loc[1] = ['wifi',0.29289940828402,'Movistar','PER_5_216:1',281.19230680474]
df.loc[2] = ['ALL|2G',1,'Entel','PER_5_216:1',960.03030000001]
df.loc[3] = ['ALL|3G',1,'Entel','PER_5_216:1',960.03030000001]
df.loc[4] = ['wifi',0.29289940828402,'Entel','PER_5_216:1',281.19230680474]
df.loc[5] = ['716.17|3G',0.29289940828402,'Entel','PER_5_216:1',281.19230680474]
# create a new column area
df["area"] = df.apply(lambda x: x["settlement_id"].split(":")[1], axis=1)
df.head()

,key,pct,brand,settlement_id,POP,area
0,ALL|2G,1,Movistar,PER_5_216:1,960.030300,1
1,wifi,0.292899,Movistar,PER_5_216:1,281.192307,1
2,ALL|2G,1,Entel,PER_5_216:1,960.030300,1
3,ALL|3G,1,Entel,PER_5_216:1,960.030300,1
4,wifi,0.292899,Entel,PER_5_216:1,281.192307,1


In [6]:
def groupRowToCol(df, new_cols, group_col, value_col_dict, fillna=0):
    # new_cols = ["Network_ID", "Node_ID"]
    # group_col = "Settlement_ID"
    # value_col_dict = {"Network_ID": "Total_Transit", "Node_ID": "Tower_Cost"}

    unique_group_col = df[group_col].unique()

    new_df = pd.DataFrame(index=unique_group_col, columns=new_cols)

    for i in range(0, len(df)):
        index_val = df[group_col][i]
        for value_col in value_col_dict:
            # if the value of value column is in new columns
            if df[value_col][i] in new_cols:
                new_df[df[value_col][i]][index_val] = df[value_col_dict[value_col]][i]
            # if value column is in new columns, directly fill in the value from original df
            elif value_col in new_cols:
                new_df[value_col][index_val] = df[value_col_dict[value_col]][i]
                
    new_df.index.name=group_col
    new_df.reset_index(inplace=True)
    new_df.fillna(value=fillna, inplace=True)
    return new_df

In [7]:
# add cvg_status col
d = {4: '4G', 3: '3G', 2: '2G', 1: 'Uncovered', 0: 'None'}
def create_new_col_names(row, is_pop=False):
    tag = '_Pct'
    if is_pop:
        tag = '_Pop'
    if row['key'] == 'wifi':
        return 'wifi'
    elif row['key'] in ['ALL|ALL', 'ALL|4G', 'ALL|3G', 'ALL|2G']:
        return "ALL_" + row['key'][-2:] + tag
    else:
        return row['brand'] + "_" + row['key'][-2:] + tag
    
def get_cvg_status(pop_type, pct, cvg_threshold=0.25):
    if pop_type == 'ALL|4G' and pct >= cvg_threshold:
        return 4
    elif pop_type == 'ALL|3G' and pct >= cvg_threshold:
        return 3
    elif pop_type == 'ALL|2G' and pct >= cvg_threshold:
        return 2
    elif pop_type in ['ALL|4G', 'ALL|3G', 'ALL|2G']:
        return 1
    else:
        return 0
    
dict = {'ALL|ALL', 'ALL|4G', 'ALL|3G', 'ALL|2G'}
# df['cvg_status'] = df.apply(lambda row: get_cvg_status(row['key'], row['pct']), axis=1) 
df["name_coverage_col"] = df.apply(lambda row: create_new_col_names(row), axis=1)
df["name_pop_col"] = df.apply(lambda row: create_new_col_names(row, True), axis=1)
type_list = ['ALL', '4G', '3G', '2G']
# i.e: ['Movistar' 'Entel']
partner_name_list = df['brand'].unique()
# new_cols example: 
#['Movistar_ALL', 'Movistar_ALL_Pop', 'Movistar_4G',...]
new_cols = list(itertools.chain.from_iterable([[a + '_' + b, a + '_' + b + '_Pop'
                ] for (a, b) in  itertools.product(
                    partner_name_list, type_list
                 )
            ]))
general_cols = list(itertools.chain.from_iterable([[a + '_' + b + '_Pct', a + '_' + b + '_Pop'
                ] for (a, b) in  itertools.product(
                    ['ALL'], type_list[1:]
                 )
            ]))
new_cols.extend(general_cols)

# add area at the end of new cols
new_cols.append("area")
# add mapping from "area" to "area"
value_col_dict={"name_coverage_col": "pct", "name_pop_col": "POP", "area": "area"}

In [8]:
df

,key,pct,brand,settlement_id,POP,area,name_coverage_col,name_pop_col
0,ALL|2G,1,Movistar,PER_5_216:1,960.030300,1,ALL_2G_Pct,ALL_2G_Pop
1,wifi,0.292899,Movistar,PER_5_216:1,281.192307,1,wifi,wifi
2,ALL|2G,1,Entel,PER_5_216:1,960.030300,1,ALL_2G_Pct,ALL_2G_Pop
3,ALL|3G,1,Entel,PER_5_216:1,960.030300,1,ALL_3G_Pct,ALL_3G_Pop
4,wifi,0.292899,Entel,PER_5_216:1,281.192307,1,wifi,wifi
5,716.17|3G,0.292899,Entel,PER_5_216:1,281.192307,1,Entel_3G_Pct,Entel_3G_Pop


In [9]:
# example 
new_df = groupRowToCol(df=df, 
              new_cols=new_cols, 
              group_col="settlement_id",
              value_col_dict=value_col_dict, fillna=0)
new_df.head()

,settlement_id,Movistar_ALL,Movistar_ALL_Pop,Movistar_4G,Movistar_4G_Pop,Movistar_3G,Movistar_3G_Pop,Movistar_2G,Movistar_2G_Pop,Entel_ALL,...,Entel_3G_Pop,Entel_2G,Entel_2G_Pop,ALL_4G_Pct,ALL_4G_Pop,ALL_3G_Pct,ALL_3G_Pop,ALL_2G_Pct,ALL_2G_Pop,area
0,PER_5_216:1,0,0,0,0,0,0,0,0,0,...,281.192307,0,0,0,0,1,960.0303,1,960.0303,1
